In [39]:
import json
import glob
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoProcessor
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

class PlancraftEnvironmentDataset(Dataset):
    def __init__(self, dataset_dir: str = "data/oracle", split="train"):
        super().__init__()
        self.split = split
        self.transform = transforms.ToTensor()
        data = []
        for example_path in sorted(glob.glob(f"{dataset_dir}/{split}/oa/*.json")):
            with open(example_path) as f:
                messages = json.load(f)
                environments = []
                for message in messages:
                    if "inventory=" in message["content"] and message["role"] == "user":
                        environments.append(self.clean((message["content"].split("\ninventory=")[-1])))
                example = {
                    "environments": environments,
                    "example_id": example_path.split("/")[-1].split(".json")[0],
                }
                data.append(example)

        print("Loading images")
        for example in data:
            example["images"] = []
            for message_idx, _ in enumerate(example["environments"]):
                img_path = f"{dataset_dir}/{split}/imgs/{example['example_id']}_{message_idx}.png"
                img = Image.open(img_path).convert("RGB")
                img = self.transform(img)
                example["images"].append(img)

        self.dataset = []
        for example in data:
            for i, (env, img) in enumerate(zip(example["environments"], example["images"])):
                self.dataset.append((env, img))

    def __len__(self) -> int:
        return len(self.dataset)
    
    @staticmethod
    def clean(s: str):
        return s.replace('"type": "', "").replace('"quantity": ', "").replace('"index": ', "").replace('"', "").replace("{", "").replace("}", "").replace(",", "").replace("[", "").replace("]", "").replace("_", " ")

    def __getitem__(self, idx: int) -> tuple:
        return self.dataset[idx]

# Load the dataset
dataset = PlancraftEnvironmentDataset(split="val")
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Load model and processor
model = AutoModel.from_pretrained("google/siglip-so400m-patch14-384")
processor = AutoProcessor.from_pretrained("google/siglip-so400m-patch14-384")

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}")
    
    for batch in progress_bar:
        texts, images = zip(*batch)
        inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs.input_ids.to(device)
        pixel_values = inputs.pixel_values.to(device)
        
        outputs = model(input_ids=input_ids, pixel_values=pixel_values)
        text_embeddings = outputs.text_embeds
        image_embeddings = outputs.image_embeds
        
        # Assuming you want to calculate contrastive loss
        logits = torch.mm(text_embeddings, image_embeddings.T)
        labels = torch.arange(len(text_embeddings)).to(device)
        loss = CrossEntropyLoss()(logits, labels) + CrossEntropyLoss()(logits.T, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({"loss": total_loss / len(progress_bar)})
        
    print(f"Epoch {epoch + 1} - Loss: {total_loss / len(dataloader)}")


AttributeError: module 'torch.library' has no attribute 'register_fake'

In [7]:




# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# texts = ["a photo of 2 cats", "a photo of 2 dogs"]
# inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")

# with torch.no_grad():
#     outputs = model(**inputs)

# logits_per_image = outputs.logits_per_image
# probs = torch.sigmoid(logits_per_image) # these are the probabilities
# print(f"{probs[0][0]:.1%} that image 0 is '{texts[0]}'")

In [12]:
len(processor.tokenizer(dataset[0][0])["input_ids"])

46

In [24]:
# inventory = [
#     {"slot": 13, "type": "stick", "quantity": 2},
#     {"slot": 20, "type": "acacia_log", "quantity": 1},
#     {"slot": 43, "type": "dead_fire_coral", "quantity": 55},
#     {"slot": 27, "type": "acacia_leaves", "quantity": 11},
#     {"slot": 28, "type": "brown_mushroom", "quantity": 23},
#     {"slot": 14, "type": "llama_spawn_egg", "quantity": 22},
#     {"slot": 45, "type": "bat_spawn_egg", "quantity": 6},
#     {"slot": 23, "type": "oak_leaves", "quantity": 8},
#     {"slot": 34, "type": "diorite_slab", "quantity": 38},
#     {"slot": 22, "type": "dark_prismarine_slab", "quantity": 54},
# ]

class TypeEmbedding(nn.Module):
    def __init__(self, model=AutoModel, tokenizer=AutoTokenizer):
        super(TypeEmbedding, self).__init__()
        self.embedding_dim = model.config.hidden_size
        self.learnable_params = nn.Parameter(torch.randn(self.embedding_dim))
        self.model = model
        self.tokenizer = tokenizer
        self.cache = {}

    def forward(self, object_types: list[str]):
        batch, new_types = ([], [])
        for object_type in object_types:
            if object_type not in self.cache:
                batch.append(object_type)
                new_types.append(object_type)
        if len(new_types) > 0:
            inputs = self.tokenizer(new_types, return_tensors="pt", padding=True)
            inputs = {k: v.cuda() for k, v in inputs.items()}
            with torch.no_grad():
                outputs = self.model(**inputs)
            for i, object_type in enumerate(new_types):
                type_embedding = outputs.last_hidden_state[i].mean(dim=0)
                self.cache[object_type] = type_embedding
        embeddings = [
            self.cache[object_type] + self.learnable_params
            for object_type in object_types
        ]
        return torch.stack(embeddings)

class InventoryEncoder(nn.Module):
    def __init__(
        self,
        model=AutoModel,
        tokenizer=AutoTokenizer,
        max_quantity=64,
        max_slot=46,
    ):
        super(InventoryEncoder, self).__init__()
        hidden_size = model.config.hidden_size
        self.type_embedding = TypeEmbedding(model, tokenizer)
        self.quantity_embedding = nn.Embedding(max_quantity, hidden_size)
        self.slot_embedding = nn.Embedding(max_slot, hidden_size)
        self.combine = nn.Linear(
            hidden_size * 3,
            hidden_size,
        )

    def forward(self, inventory: list[dict]):
        type_embeddings = self.type_embedding([item["type"] for item in inventory])
        quantities = torch.tensor(
            [item["quantity"] for item in inventory], dtype=torch.long
        )
        slots = torch.tensor([item["slot"] for item in inventory], dtype=torch.long)

        quantities = quantities.cuda()
        slots = slots.cuda()

        quantity_embeddings = self.quantity_embedding(quantities)
        slot_embeddings = self.slot_embedding(slots)
        x_concat = torch.cat(
            [type_embeddings, quantity_embeddings, slot_embeddings], dim=-1
        )
        embed = self.combine(x_concat).mean(dim=0)
        return embed


encoder = InventoryEncoder(model, tokenizer)
encoder = encoder.cuda()

In [31]:
class InventoryGenerator(nn.Module):
    def __init__(
        self,
        model=AutoModel,
        tokenizer=AutoTokenizer,
        max_quantity=64,
        max_slot=46,
    ):
        super(InventoryGenerator, self).__init__()
        hidden_size = model.config.hidden_size
        self.max_quantity = max_quantity
        self.max_slot = max_slot
        self.hidden_size = hidden_size

        self.fc = nn.Linear(hidden_size, hidden_size * 3)
        self.type_decoder = TypeEmbedding(model, tokenizer)
        self.quantity_decoder = nn.Linear(hidden_size, max_quantity)
        self.slot_decoder = nn.Linear(hidden_size, max_slot)

    def forward(self, inventory_embedding):
        x = self.fc(inventory_embedding)

        type_embeds, quantity_embeds, slot_embeds = torch.split(
            x, self.hidden_size, dim=-1
        )

        # Decode type embeddings
        decoded_types = self.type_decoder.decode(type_embeds)

        # Decode quantity and slot embeddings
        quantities = self.quantity_decoder(quantity_embeds)
        slots = self.slot_decoder(slot_embeds)

        # Convert logits to indices
        quantities = torch.argmax(quantities, dim=-1)
        slots = torch.argmax(slots, dim=-1)

        # Create the decoded inventory list
        decoded_inventory = []
        for obj_type, quantity, slot in zip(decoded_types, quantities, slots):
            decoded_inventory.append(
                {
                    "type": obj_type,
                    "quantity": quantity.item(),
                    "slot": slot.item(),
                }
            )

        return decoded_inventory


# Example of how to use the InventoryEmbedding and InventoryGenerator
class InventoryAutoencoder(nn.Module):
    def __init__(
        self,
        model=AutoModel,
        tokenizer=AutoTokenizer,
        max_quantity=64,
        max_slot=46,
    ):
        super(InventoryAutoencoder, self).__init__()
        self.encoder = InventoryEncoder(model, tokenizer, max_quantity, max_slot)
        self.decoder = InventoryGenerator(model, tokenizer, max_quantity, max_slot)

    def forward(self, inventory: list[dict]):
        encoded = self.encoder(inventory)
        decoded = self.decoder(encoded)
        return decoded


# Example usage
inventory = [
    {"slot": 13, "type": "stick", "quantity": 2},
    {"slot": 20, "type": "acacia_log", "quantity": 1},
    {"slot": 43, "type": "dead_fire_coral", "quantity": 55},
    {"slot": 27, "type": "acacia_leaves", "quantity": 11},
    {"slot": 28, "type": "brown_mushroom", "quantity": 23},
    {"slot": 14, "type": "llama_spawn_egg", "quantity": 22},
    {"slot": 45, "type": "bat_spawn_egg", "quantity": 6},
    {"slot": 23, "type": "oak_leaves", "quantity": 8},
    {"slot": 34, "type": "diorite_slab", "quantity": 38},
    {"slot": 22, "type": "dark_prismarine_slab", "quantity": 54},
]

autoencoder = InventoryAutoencoder(model, tokenizer)
autoencoder = autoencoder.to("cuda")
encoded_inventory = autoencoder.encoder(inventory)

In [ ]:
autoencoder.decoder(encoded_inventory)


In [8]:
import json

with open("data/train.json") as f:
    data = json.load(f)

In [25]:
encoder(data[0]["slotted_inventory"])


tensor([ 0.5024,  0.7074,  1.2459,  ...,  0.2150, -0.3527, -0.0361],
       device='cuda:0', grad_fn=<MeanBackward1>)

In [ ]:
# load data/train.json

# with open('data/train.json') as f:
#     data = json.load(f)